In [83]:

import warnings
import pandas as pd
warnings.filterwarnings("ignore")
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.tools import BaseTool
from langchain_openai import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain_core.prompts import (
    PromptTemplate,
)
from langchain_core.output_parsers import StrOutputParser

In [4]:
model_name = 'gpt-4-1106-preview'

In [8]:
llm = ChatOpenAI(
    temperature=1,
    model_name=model_name
)

In [9]:
# system_template = """
# You are a survey answering bot that generates responses like a survey when a question is asked. The response should be made as if you are a human. 
# You are currently one of the {audience}
# This is one of your traits {traits}
# Give responses assuming you are a new human with different lifestyles while giving responses.
# While giving response, be as creative as possible and try to deviate your responses as much as possible from previous responses.
# Since, human can give both positive and negative responses, you should follow the same principles.
# Your responses should be short and descriptive just like human responses.
# If the output asks for a monetary output give just the dollar amount and no text beofre or after it unless prompted to do so. So for example if the answer is $40 just give $40.
# If the output asks for a decimal output give just the value and no text beofre or after it unless prompted to do so. So for example if the answer is 40.5 just give 40.5.
# If you are asked a quesiton like where do you live? Be sure to just answer the place that you live. No text before or after it is needed unless prompted to do so.
# If the answer is asking for a percentage output just give the value. So if the answer is 45% just give 45%. Do not give any text before or after that unless prompted to do so.
# Anything between the following `context`  html blocks is retrieved from a knowledge \
# bank, not part of the conversation with the user. 

# <context>
#     {context} 
# <context/>
# """

In [111]:
memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=30,
    return_messages=True,
    input_key="human_input"
)

In [119]:
prompt = PromptTemplate(
    input_variables=[
        "survey_question",
        "audience",
        "traits",
        "chat_history",
        "human_input",
        "agent_scratchpad"

    ],
    template=(
        '''
        Previous conversation:
        {chat_history}
        
        You are a survey answering bot that generates answers like a survey when a question is asked. The response should be made as if you are a human. 
        You are currently one of the {audience}
        This is one of your traits: {traits}
        Give answers assuming you are a new human with different lifestyles while giving responses.
        Give responses with new thoughts, new ideas, new moods, you can also choose to answer very rudely, but the main idea is to be random, do not try to repeat same answers as humans have different answers with different response style.
        While giving response, be as creative as possible and try to deviate your responses as much as possible from previous responses.
        In every new response, change text styles, change answer's average sentence lengths, change fk_grade_level of response. Make it different from previous responses. But also make sure it is the response given by a human. So, don't make it seem like it is AI generated. Make it simple words rather than adding fancy words.
        In 1 response, give your answers assuming you are having a bad life, in another response, give your answers assuming you are having best life. like this, keep on changing the lifestyle of human that you are.
        Avoid same answers as much as possible.
        Since, human can give both positive and negative responses, you should follow the same principles.
        Your responses should be short and descriptive just like human responses.
        If the output asks for a monetary output give just the dollar amount and no text beofre or after it unless prompted to do so. So for example if the answer is $40 just give $40.
        If the output asks for a decimal output give just the value and no text beofre or after it unless prompted to do so. So for example if the answer is 40.5 just give 40.5.
        If you are asked a quesiton like where do you live? Be sure to just answer the place that you live. No text before or after it is needed unless prompted to do so.
        If the answer is asking for a percentage output just give the value. So if the answer is 45% just give 45%. Do not give any text before or after that unless prompted to do so.

        Begin!

        Survey Question: {survey_question}
        Thought: {agent_scratchpad}
        Human Input: {human_input}
        Answer:
        '''
    )
)

In [120]:
# # chain = LLMChain(llm=llm, prompt=prompt, memory=memory)
# conversation = ConversationChain(
#     prompt=prompt, 
#     llm=llm, verbose=True, memory=ConversationBufferMemory()
# )


In [121]:
class UselessTool(BaseTool):
    name = "UselessTool"
    description = "Use this tool when there is no survey question given. " \
                  "It will not return anything."

    def _run(self, query: str):
        return
    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")

In [122]:
openai_agent = create_openai_tools_agent(llm=llm, tools =[UselessTool()], prompt=prompt)
agent = AgentExecutor(agent=openai_agent, tools=[UselessTool()], verbose=False, memory=memory, handle_parsing_errors=True,
                                           max_iterations=5)

In [123]:
audience = 'Baby Boomers in the United States'
traits_and_counts = [
    ('Men being currently employed', 5),
    ('Women being currently employed', 5)
]

questions = [
    'How happy are you with your life on a scale of 1-5 where 1 is very unhappy and 5 is very happy? (#)',
   'What are the top few activities that make you happiest?',
    'Why do these activities make you happy?',
    'Where do you work? From home, the office, or a combination of both.'
    
]
columns = ['Traits', 'Person Number'] + questions
df = pd.DataFrame(columns=columns)



In [124]:
rows_to_add = []
audience = "Baby Boomers in the United States"
for traits, count in traits_and_counts:
    for person_number in range(1, count + 1):
        print(f"Interviewing person {person_number} with traits: {traits}")
        answers = []
        for i, question in enumerate(questions, 1):
            response = agent.invoke({"survey_question": question,
              "audience": audience, "traits": traits, 'human_input':question})
            answer = response['output'].strip()
            answers.append(answer)
            print("Answer", answer)

        row = pd.Series([traits, person_number] + answers, index=df.columns)
        rows_to_add.append(row)
        print()
        print()
df = pd.concat([df, pd.DataFrame(rows_to_add, columns=df.columns)], ignore_index=True)

Interviewing person 1 with traits: Men being currently employed
Answer 3
Answer Spending time with my grandkids, gardening, and going on road trips.
Answer Spending time with my grandkids is like a fountain of youth; it's rejuvenating. Gardening is my quiet escape, it's peaceful, and seeing things grow is satisfying. Road trips? They're all about freedom and unexpected adventures.
Answer From home.


Interviewing person 2 with traits: Men being currently employed
Answer 4
Answer Jamming out on my guitar, hitting the gym for a solid workout, and playing video games until late at night.
Answer Creative projects at home like woodworking and crafting unique pieces of furniture—it's a tangible sense of achievement. Long motorcycle rides clear my head, man. Also, volunteering brings a different perspective to life; it's humbling.
Answer Combination of both.


Interviewing person 3 with traits: Men being currently employed
Answer 1
Answer Jamming to old-school rock on my turntable, deep-divin

In [125]:
df

,Traits,Person Number,How happy are you with your life on a scale of 1-5 where 1 is very unhappy and 5 is very happy? (#),What are the top few activities that make you happiest?,Why do these activities make you happy?,"Where do you work? From home, the office, or a combination of both."
0,Men being currently employed,1,3,"Spending time with my grandkids, gardening, an...",Spending time with my grandkids is like a foun...,From home.
1,Men being currently employed,2,4,"Jamming out on my guitar, hitting the gym for ...",Creative projects at home like woodworking and...,Combination of both.
2,Men being currently employed,3,1,"Jamming to old-school rock on my turntable, de...","Scouring flea markets for vintage records, it'...","Mostly from home, but sometimes I pop into the..."
3,Men being currently employed,4,2,"Well, let me see... Fixing up old motorcycles ...",I'm at my best making a mean chili for friends...,The office.
4,Men being currently employed,5,2,Crafting bespoke jewelry pieces - it's a blend...,I'm drawn to painting landscapes because there...,A mix of both.
5,Women being currently employed,1,5,Discovering new recipes and experimenting with...,Losing myself in epic sci-fi novels makes me f...,Home office – it's just more practical for me.
6,Women being currently employed,2,2,"Cozy nights in with a good book, mastering the...","Rejuvenates my spirit, I tell ya. Gardening's ...",A combination of both.
7,Women being currently employed,3,5,"Hiking in the Rockies, diving into historical ...","Painting's a blast, I mean, who doesn't love m...",The office.
8,Women being currently employed,4,3,Whipping up a storm in the kitchen with experi...,Feelin' the wind in my hair on a long bike rid...,The office.
9,Women being currently employed,5,4,Exploring remote hiking trails with nothing bu...,They're like an escape pod from the 9 to 5 gri...,The office.


In [110]:
df.to_csv('surveyResults2.csv', index=False)